In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/Colab\ Notebooks/mest

/content/drive/MyDrive/Colab Notebooks/mest


In [3]:
from tqdm import tqdm
import os, re
import requests
import random

import numpy as np
import matplotlib.pyplot as plt
from copy import copy

from torchvision import transforms
import torch
from torchvision.utils import save_image

from PIL import Image, ImageOps
import PIL

# Normalize pos and neg

In [4]:
!mkdir "bin/0res"
!mkdir "bin/1res"

In [5]:
DATA_POS = "bin/11"
DATA_NEG = "bin/00"

def open_image(image):
    # чтобы изображение не поворачивалось на 90 градусов, если w < h
    # https://stackoverflow.com/questions/4228530/pil-thumbnail-is-rotating-my-image
    img = Image.open(image)
    img = ImageOps.exif_transpose(img)
    return img

def path_dir(dir):
    paths = []
    for p in sorted(os.listdir(dir)):
        path = os.path.join(dir, p)
        paths.append(path)
    return paths

def file_name(path):
    p = re.split(r"/", path)
    return p[-1]

def plot_image(path, transform, save_dir, resize=None, title = None):
    f = Image.open(path)

    if resize:
        score = resize_score(f, resize)
        transform = copy(transform)
        transform.transforms = [transforms.Resize(score)] + transform.transforms
        # print(transform)

    image = transform(f)
    save_image(image, f"{save_dir}/{file_name(path)}")
    # image = image.permute(1, 2, 0)
    # if title:
    #     title = title + f":  {file_name(path)}      {tuple(image.shape)}"
    # else:
    #     title = f"{file_name(path)}      {tuple(image.shape)}"

    # plt.title(title)
    # plt.imshow(image)


def resize_score(image, coef):
    w, h = image.size
    m = min(h, w)
    return int(m * coef)


def plot_transformed_images(pos_dir, neg_dir):
    pos_paths = path_dir(pos_dir)
    neg_paths = path_dir(neg_dir)

    # trans_pos = transforms.Compose([
    #     # transforms.RandomCrop(500),
    #     # transforms.CenterCrop(700),
    #     transforms.ToTensor()])

    trans_pos = transforms.Compose([
            # transforms.CenterCrop(500),
            transforms.ToTensor()])

    trans_neg = transforms.Compose([
        transforms.Resize(1000),
        # transforms.CenterCrop(500),
        transforms.ToTensor()])

    num = len(pos_paths)
    # neg_paths = random.sample(neg_paths, k=num)
    # neg_paths = neg_paths[:num]

    resize = [
      0.8, 0.8, 0.8, 0.2,   1,   1,   1,   1, 0.5,   1,
      0.6, 0.5, 0.5, 0.8, 0.8, 0.8, 0.6, 0.6, 0.5, 0.5,
      0.8, 0.8,   1, 0.8, 0.8
    ]

    for i in range(num):
        # plt.figure(figsize=(15, 6))
        # plt.subplot(1, 2, 1)
        plot_image(pos_paths[i], trans_pos, save_dir="bin/1res", resize=resize[i], title = f"{i+1} - {resize[i]}")

        # plt.subplot(1, 2, 2)
        # plot_image(neg_paths[i], trans_neg, save_dir="data/0_resize")
        # plt.show()

    for i in range(len(neg_paths)):
        plot_image(neg_paths[i], trans_neg, save_dir="bin/0res")

In [6]:
plot_transformed_images(DATA_POS, DATA_NEG)

In [7]:
def info(dir):
    image_paths = []
    transform = transforms.ToTensor()

    for i, p in enumerate(sorted(os.listdir(dir)[:30])):
        p = os.path.join(dir, p)
        image_paths.append(p)
        img = open_image(p)
        transformed_image = transform(img)
        # {os.path.getsize(p) // (10**6)} {transformed_image.shape}
        w, h = img.size
        print(f"{i:2}:\t{p:50} \t {h} x {w} \t")

In [8]:
info("bin/11")

 0:	bin/11/1-Протасовым (хороший)-1!.png              	 2771 x 1374 	
 1:	bin/11/1-Протасовым (хороший)-2!.png              	 2791 x 2124 	
 2:	bin/11/1-Протасовым (хороший)-3!.png              	 1164 x 2236 	
 3:	bin/11/1-Тургеневу (обычный)!.png                 	 4821 x 7217 	
 4:	bin/11/2-Александре (идеальный)-11!.png           	 1438 x 1380 	
 5:	bin/11/2-Александре (идеальный)-22!.png           	 958 x 1360 	
 6:	bin/11/2-Александре (идеальный)-33!.png           	 1406 x 1380 	
 7:	bin/11/2-Александре (идеальный)-44!.png           	 1406 x 1360 	
 8:	bin/11/2-Александре (идеальный)-55!.png           	 2742 x 2680 	
 9:	bin/11/2-Александре (идеальный)-66!.png           	 1416 x 1380 	
10:	bin/11/2-Воейковой (обычный)!.png               	 848 x 2052 	
11:	bin/11/2-Вяземскому (плохой)!.png                 	 765 x 902 	
12:	bin/11/2-Неизвестной (плохой)!.png               	 1488 x 1357 	
13:	bin/11/3-Булгакову (обычный)!.png                 	 1162 x 1500 	
14:	bin/11

In [10]:
info("bin/1res")

 0:	bin/1res/1-Протасовым (хороший)-1!.png            	 2216 x 1099 	
 1:	bin/1res/1-Протасовым (хороший)-2!.png            	 2232 x 1699 	
 2:	bin/1res/1-Протасовым (хороший)-3!.png            	 931 x 1788 	
 3:	bin/1res/1-Тургеневу (обычный)!.png               	 964 x 1443 	
 4:	bin/1res/2-Александре (идеальный)-11!.png         	 1438 x 1380 	
 5:	bin/1res/2-Александре (идеальный)-22!.png         	 958 x 1360 	
 6:	bin/1res/2-Александре (идеальный)-33!.png         	 1406 x 1380 	
 7:	bin/1res/2-Александре (идеальный)-44!.png         	 1406 x 1360 	
 8:	bin/1res/2-Александре (идеальный)-55!.png         	 1371 x 1340 	
 9:	bin/1res/2-Александре (идеальный)-66!.png         	 1416 x 1380 	
10:	bin/1res/2-Воейковой (обычный)!.png             	 508 x 1229 	
11:	bin/1res/2-Вяземскому (плохой)!.png               	 382 x 450 	
12:	bin/1res/2-Неизвестной (плохой)!.png             	 743 x 678 	
13:	bin/1res/3-Булгакову (обычный)!.png               	 929 x 1199 	
14:	bin/1res/3-

In [9]:
info("bin/00")

 0:	bin/00/122-1.png                                   	 1000 x 750 	
 1:	bin/00/122-10.png                                  	 1000 x 750 	
 2:	bin/00/122-11.png                                  	 1000 x 750 	
 3:	bin/00/122-12.png                                  	 1000 x 750 	
 4:	bin/00/122-2.png                                   	 1000 x 750 	
 5:	bin/00/122-3.png                                   	 1000 x 750 	
 6:	bin/00/122-4.png                                   	 1000 x 750 	
 7:	bin/00/122-5.png                                   	 1000 x 750 	
 8:	bin/00/122-6.png                                   	 1000 x 750 	
 9:	bin/00/122-7.png                                   	 1000 x 750 	
10:	bin/00/122-8.png                                   	 1000 x 750 	
11:	bin/00/122-9.png                                   	 1000 x 750 	
12:	bin/00/142-1.png                                   	 1000 x 750 	
13:	bin/00/142-10.png                                  	 1000 x 750 	
14:	bin/00/142-11.pn

In [11]:
info("bin/0res")

 0:	bin/0res/122-1.png                                 	 1333 x 1000 	
 1:	bin/0res/122-10.png                                	 1333 x 1000 	
 2:	bin/0res/122-11.png                                	 1333 x 1000 	
 3:	bin/0res/122-12.png                                	 1333 x 1000 	
 4:	bin/0res/122-2.png                                 	 1333 x 1000 	
 5:	bin/0res/122-3.png                                 	 1333 x 1000 	
 6:	bin/0res/122-4.png                                 	 1333 x 1000 	
 7:	bin/0res/122-5.png                                 	 1333 x 1000 	
 8:	bin/0res/122-6.png                                 	 1333 x 1000 	
 9:	bin/0res/122-7.png                                 	 1333 x 1000 	
10:	bin/0res/122-8.png                                 	 1333 x 1000 	
11:	bin/0res/122-9.png                                 	 1333 x 1000 	
12:	bin/0res/142-1.png                                 	 1333 x 1000 	
13:	bin/0res/142-10.png                                	 1333 x 1000 	
14:	bi